In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import project_lib as mylib
from mpl_toolkits.mplot3d.axes3d import Axes3D
import project_lib as mylib
from scipy.interpolate import griddata
from matplotlib import cm
%matplotlib notebook

In this file I want to create the energy terrain graphs for a 2 qubit system

In [ ]:
# Twice as wide as it is tall.
fig = plt.figure(figsize=plt.figaspect(0.5))

#---- First subplot
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)
surf1 = ax1.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax1.set_zlim3d(-1.01, 1.01)

#---- Second subplot
ax2 = fig.add_subplot(1, 2, 2, projection='3d',sharex = ax1,sharey = ax1,sharez = ax1)
surf2 = ax2.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

plt.show()

#### In the next part I would like to be able to plot a series of energy level sheets so that I can look at the separation of the energy levels between multiple levels

In [ ]:
######################################################
#         LOAD DATA
######################################################
BETWEENLEVELS = (1,None)

DATA = mylib.load_object("Nicks_device_square_(radial_energy_levels)(v001)")
thetas = range(1,100)
ss = np.linspace(0,1,1000)
x = np.asarray([(1-ss) if theta<= 45 else (1-ss)*np.tan((90-theta)*np.pi/180.) for theta in thetas]) 
y = np.asarray([(1-ss)*np.tan((theta)*np.pi/180.)*(1-ss) if theta <=45 else (1-ss) for theta in thetas])
energy_levels = DATA['energy_levels']
if BETWEENLEVELS[1]==None:
    z = [energy_levels[theta][:,BETWEENLEVELS[0]] for theta in range(0,99)]
else:
    z = [energy_levels[theta][:,BETWEENLEVELS[1]]- energy_levels[theta][:,BETWEENLEVELS[0]] for theta in range(0,99)]
z = np.asarray(z)

x = np.reshape(x,x.size)
y = np.reshape(y,y.size)
z = np.reshape(z,z.size)

xi = np.linspace(0,1,100)
yi = np.linspace(0,1,100)
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')


# Twice as wide as it is tall.
fig = plt.figure(figsize=plt.figaspect(0.5))

#---- First subplot
ax1 = fig.add_subplot(1, 1, 1, projection='3d')
X, Y = np.meshgrid(xi, yi)
Z = zi
surf1 = ax1.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False,vmin =min(z),vmax=max(z))
ax1.set_xlabel('$A_{log}(s)$')
ax1.set_ylabel('$A_{anc}(s)$')
ax1.set_zlabel('E')
#ax1.set_zlim3d(-1.01, 1.01)
plt.show()

In [ ]:
BR = [cm.Blues,cm.Reds]

######################################################
#         LOAD DATA
######################################################
LEVELS = [0,1,2]

DATA = mylib.load_object("Nicks_device_square_(radial_energy_levels)(v001)")
thetas = range(1,100)
ss = np.linspace(0,1,1000)
x = np.asarray([(1-ss) if theta<= 45 else (1-ss)*np.tan((90-theta)*np.pi/180.) for theta in thetas]) 
y = np.asarray([(1-ss)*np.tan((theta)*np.pi/180.)*(1-ss) if theta <=45 else (1-ss) for theta in thetas])
energy_levels = DATA['energy_levels']

x = np.reshape(x,x.size)
y = np.reshape(y,y.size)
xi = np.linspace(0,1,100)
yi = np.linspace(0,1,100)

#calculate the minimum energy involved
z0 = [energy_levels[theta][:,min(LEVELS)] for theta in range(0,99)]
z0 = np.asarray(z0)
z0 = np.reshape(z0,z0.size)
minE=min(z0)

# Twice as wide as it is tall.
fig = plt.figure(figsize=plt.figaspect(0.5))

#---- First subplot
ax1 = fig.gca(projection='3d')
X, Y = np.meshgrid(xi, yi)
surfs = []
for i in range(1,len(LEVELS)):
    z = [energy_levels[theta][:,LEVELS[i]] for theta in range(0,99)]
    z = np.asarray(z)
    z = np.log(np.reshape(z,z.size)-z0)
    zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')
    Z = zi
    surfs.append(ax1.plot_surface(X, Y, Z, cmap=BR[i%2], linewidth=0.1, alpha=1., antialiased=True,vmin =-14,vmax=max(z)))
    
ax1.set_xlabel('$A_{log}(s)$')
ax1.set_ylabel('$A_{anc}(s)$')
ax1.set_zlabel('$\log(E-E_{%s})$'%min(LEVELS))
#ax1.set_zlim3d(-1.01, 1.01)
plt.show()

In [ ]:
min(z[z!=-np.inf])